In [89]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso


#評価用

train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

with open('ensamble/rf_train.pickle', mode='br') as fi:
    train_p1 = pickle.load(fi)
    
with open('ensamble/lgb_train.pickle', mode='br') as fi:
    train_p2 = pickle.load(fi)





#データ読み込み
with open('ensamble/rf_test.pickle', mode='br') as fi:
    test_p1 = pickle.load(fi)
    
with open('ensamble/lgb_test.pickle', mode='br') as fi:
    test_p2 = pickle.load(fi)

## アンサンブル

In [90]:
df_train = pd.concat([train['Outcome'],train_p1['pred'],train_p2['pred']], axis=1, ignore_index=True).rename(columns={0: 'true', 1: 'pred1', 2: 'pred2'})
df_train['ensemble'] = df_train['pred1']*0.5 + df_train['pred2']*0.5

df_test = pd.concat([test['index'],test_p1['pred'],test_p2['pred']], axis=1, ignore_index=True).rename(columns={0: 'index', 1: 'pred1', 2: 'pred2'})
df_test['ensemble'] = df_test['pred1']*0.5 + df_test['pred2']*0.5

def evaluate_ensemble(input_df, col_pred):
    print('[auc] model1:{:.4f}, model2:{:.4f}, -> emsamble]{:.4f}'.format(
        roc_auc_score(input_df['true'], input_df['pred1']),
        roc_auc_score(input_df['true'], input_df['pred2']),
        roc_auc_score(input_df['true'], input_df[col_pred]),
    ))

In [91]:
evaluate_ensemble(df_train, col_pred='ensemble')

[auc] model1:0.8053, model2:0.8182, -> emsamble]0.8192


In [93]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_emsamble.csv', index=None, header=False,)

## スタッキング

In [ ]:
x, y = df_train[['pred1', 'pred2',]], df_train[['true']]
oof = np.zero(len(x))
models = []

cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x,y))
for nfold in np.arrange(5):
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x.loc[idx_tr, :], y.loc[idx_tr, :]
    x_va, y_va = x.loc[idx_va, :], y.loc[idx_tr, :]
    
    model = Lasso(alpha=0.01)
    model.fit(x_tr, y_tr)
    models.append(model)
    
    y_va_pred = model.predict(x_va)
    oof[idx_va] = y_va_pred
    
df_train['pred_ensemble2'] = oof
df_train['pred_ensemble2'] = df_train['pred_ensemble2'].clip(lower=0, upper=1)
df_train.head()

In [ ]:
evaluate_ensemble(df_train, col_pred='pred_ensemble2')

In [ ]:
de_test['pred_emsemble'] = 0
for model in models:
    df_test['pred_emsemble2'] += model.predict(df_test[['pred1', 'pred2']]) / len(models)
    df_test['pred_emsemble2'] df_test['pred_emsemble2'].clip(lower0, upper=1)
    evaluate_emsemble(df_test, col_pred='pred_ensemble2')